In [1]:
!pip install langchain chromadb sentence-transformers pdfplumber python-docx openai tiktoken



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from docx import Document
import pdfplumber

def read_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])

def read_docx(file_path):
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(raw_text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", " "]
    )
    return splitter.split_text(raw_text)


In [16]:
sample_text='The answers you need—right at your finger tips. Cut through the clutter and get straight to credible, up-to-date answers. This free app syncs across devices ..'

In [17]:
response=chunk_text(sample_text)

In [18]:
response

['The answers you need—right at your finger tips. Cut through the clutter and get straight to credible, up-to-date answers. This free app syncs across devices ..']

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

def generate_embeddings(chunks):
    return model.encode(chunks, show_progress_bar=True)


C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.create_collection("hr_policy")

def add_to_vector_db(chunks, embeddings):
    for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
        collection.add(
            documents=[chunk],
            embeddings=[emb.tolist()],
            ids=[f"chunk_{i}"]
        )


In [20]:
def get_top_chunks(query, k=3):
    query_emb = model.encode([query])[0]
    results = collection.query(
        query_embeddings=[query_emb.tolist()],
        n_results=k
    )
    return results["documents"][0]


In [21]:
from dotenv import load_dotenv

In [22]:
load_dotenv()

True

In [25]:
import requests

GROQ_API_KEY = os.getenv("GROQ_API_KEY")  # Replace with your key

def generate_response_with_groq(query, top_chunks):
    context = "\n\n".join(top_chunks)

    system_prompt = "You are an expert HR assistant. Use the provided HR policy context to accurately answer the user’s question. Always cite the policy sections if applicable."

    payload = {
        "model": "llama3-70b-8192",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ],
        "temperature": 0.3
    }

    response = requests.post(
        url="https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {GROQ_API_KEY}",
            "Content-Type": "application/json"
        },
        json=payload
    )

    return response.json()['choices'][0]['message']['content']


In [33]:
# 1. Load a document
text = read_pdf(r"C:\Users\harsh\Downloads\MCP.pdf")  # or read_docx/read_txt

# 2. Chunk it
chunks = chunk_text(text)

# 3. Embed chunks
embeddings = generate_embeddings(chunks)


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.46it/s]


In [34]:
embeddings

array([[-9.1101266e-03, -4.8343115e-03,  1.1705715e-04, ...,
        -4.8828423e-02, -1.1610587e-01,  3.7160467e-02],
       [-3.7461881e-02, -2.4240071e-02,  2.7802402e-02, ...,
         1.6751888e-03,  1.3276938e-02,  4.9852837e-02],
       [-7.5757192e-03, -4.5640945e-02,  1.1800321e-02, ...,
         3.0081499e-02, -9.2213802e-02,  1.8424278e-02],
       ...,
       [-7.3035844e-02, -5.3828001e-02, -1.2783484e-02, ...,
         1.3071555e-01,  2.9443709e-02, -6.0335286e-02],
       [-2.9477840e-02, -3.5294946e-02, -3.8567547e-02, ...,
         7.9272062e-02,  1.0463837e-02, -8.7959565e-02],
       [-2.6764071e-02, -4.8882872e-02, -6.0507175e-02, ...,
         7.8133449e-02,  3.6813398e-03,  4.4863503e-03]], dtype=float32)

In [35]:
add_to_vector_db(chunks, embeddings)

# 5. Ask a question
query = "How many vacation days do I get?"
top_chunks = get_top_chunks(query)

In [36]:
top_chunks

['look like:\n12\nDailyDoseofDS.com\nThis Python function, registered with @mcp.tool(), can be invoked by the AI via\nMCP.\nWhen the AI calls tools/call with name "get_weather" and {"location": "San\nFrancisco"} as arguments, the server will execute get_weather("San Francisco")\nand return the dictionary result.\nThe client will get that JSON result and make it available to the AI. Notice the\ntool returns structured data (temperature, conditions), and the AI can then use or',
 'do, the Client knows how to speak MCP to actually carry out those instructions\nwith the server.\n10\nDailyDoseofDS.com\nServer\nThe MCP Server is the external program or service that actually provides the\ncapabilities (tools, data, etc.) to the application.\nAn MCP Server can be thought of as a wrapper around some functionality, which\nexposes a set of actions or resources in a standardized way so that any MCP\nClient can invoke them.',
 'F\n2025 EDITION R\nE\nE\nM C P\nTHE ILLUSTRATED\nGUIDEBOOK\nDaily Dose 

In [ ]:

# 4. Store in Vector DB


# 6. Generate the final response
response = generate_response(query, top_chunks)
print(response)
